In [ ]:
#triple riding

import cv2
import numpy as np
from ultralytics import YOLO
import torch

class TripleRiderDetector:
    def __init__(self, model_path="yolov8x.pt"):
        # Initialize YOLOv8 model - using larger model for better accuracy
        self.model = YOLO(model_path)
        
        # Class IDs for motorcycle and person (standard COCO classes)
        self.MOTORCYCLE_CLASS = 3  # motorcycle class id
        self.PERSON_CLASS = 0      # person class id
        
        # IoU threshold for associating persons with motorcycles
        self.IOU_THRESHOLD = 0.2
        
    def calculate_iou(self, box1, box2):
        """Calculate IoU between two bounding boxes"""
        x1 = max(box1[0], box2[0])
        y1 = max(box1[1], box2[1])
        x2 = min(box1[2], box2[2])
        y2 = min(box1[3], box2[3])
        
        intersection = max(0, x2 - x1) * max(0, y2 - y1)
        area1 = (box1[2] - box1[0]) * (box1[3] - box1[1])
        area2 = (box2[2] - box2[0]) * (box2[3] - box2[1])
        
        union = area1 + area2 - intersection
        return intersection / union if union > 0 else 0
    
    def expand_motorcycle_box(self, box, expand_factor=1.2):
        """Expand motorcycle bounding box to better capture riders"""
        center_x = (box[0] + box[2]) / 2
        center_y = (box[1] + box[3]) / 2
        width = (box[2] - box[0]) * expand_factor
        height = (box[3] - box[1]) * expand_factor
        
        return [
            max(0, center_x - width/2),
            max(0, center_y - height/2),
            center_x + width/2,
            center_y + height/2
        ]
    
    def process_frame(self, frame):
        """Process a single frame to detect triple riding"""
        # Run YOLOv8 detection
        results = self.model(frame)
        
        # Initialize lists for motorcycles and persons
        motorcycles = []
        persons = []
        
        # Extract detections
        for result in results:
            boxes = result.boxes
            for box in boxes:
                # Get class id and confidence
                cls = int(box.cls[0])
                conf = float(box.conf[0])
                
                # Only consider high confidence detections
                if conf < 0.5:
                    continue
                
                # Get bounding box coordinates
                x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
                bbox = [x1, y1, x2, y2]
                
                if cls == self.MOTORCYCLE_CLASS:
                    motorcycles.append(bbox)
                elif cls == self.PERSON_CLASS:
                    persons.append(bbox)
        
        # Process each motorcycle
        for motorcycle_idx, motorcycle_box in enumerate(motorcycles):
            # Expand motorcycle box to better capture riders
            expanded_box = self.expand_motorcycle_box(motorcycle_box)
            
            # Count additional riders associated with this motorcycle
            additional_riders = 0
            for person_box in persons:
                if self.calculate_iou(expanded_box, person_box) > self.IOU_THRESHOLD:
                    additional_riders += 1
            
            # Total riders is at least 1 (the driver) plus any additional detected riders
            total_riders = max(1, additional_riders)
            
            # Draw bounding box and text
            color = (0, 255, 0)  # Default green
            text = f"Riders: {total_riders}"
            
            if total_riders >= 3:
                color = (0, 0, 255)  # Red for triple riding
                text = "TRIPLE RIDING DETECTED!"
                
            # Draw motorcycle bounding box
            cv2.rectangle(frame, 
                         (int(motorcycle_box[0]), int(motorcycle_box[1])),
                         (int(motorcycle_box[2]), int(motorcycle_box[3])),
                         color, 2)
            
            # Add text
            cv2.putText(frame, text,
                       (int(motorcycle_box[0]), int(motorcycle_box[1] - 10)),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)
        
        return frame
    
    def process_video(self, input_path, output_path):
        """Process video file"""
        cap = cv2.VideoCapture(input_path)
        
        # Get video properties
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        fps = int(cap.get(cv2.CAP_PROP_FPS))
        
        # Initialize video writer
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
        
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            
            # Process frame
            processed_frame = self.process_frame(frame)
            
            # Write frame
            out.write(processed_frame)
            
            # Display frame (optional)
            cv2.imshow('Frame', processed_frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        
        # Release resources
        cap.release()
        out.release()
        cv2.destroyAllWindows()

def main():
    # Initialize detector
    detector = TripleRiderDetector()
    
    # Process video
    input_video = "C:/Users/DELL/Desktop/accident detection/numberplate/vid5.mp4"  # Replace with your video path
    output_video = "C:/Users/DELL/Desktop/accident detection/numberplate/triple5.mp4"
    
    detector.process_video(input_video, output_video)

if __name__ == "__main__":
    main()


In [ ]:
#triple riding yolov8x
import cv2
import numpy as np
from ultralytics import YOLO
import torch
from collections import defaultdict
import time

class TripleRiderDetector:
    def __init__(self, model_path="yolov8x.pt"):
        # Initialize YOLOv8 model (using the largest model for best accuracy)
        self.model = YOLO(model_path)
        
        # Constants
        self.MOTORCYCLE_CLASS = 3  # COCO class for motorcycle
        self.PERSON_CLASS = 0      # COCO class for person
        self.CONFIDENCE_THRESHOLD = 0.5
        self.IOU_THRESHOLD = 0.3
        self.TRACKING_HISTORY = defaultdict(list)
        self.MOTORCYCLE_RIDERS = defaultdict(int)
        self.DETECTION_PERSISTENCE = 5  # frames
        
    def calculate_overlap(self, box1, box2):
        """Calculate IoU between two bounding boxes"""
        x1 = max(box1[0], box2[0])
        y1 = max(box1[1], box2[1])
        x2 = min(box1[2], box2[2])
        y2 = min(box1[3], box2[3])
        
        intersection = max(0, x2 - x1) * max(0, y2 - y1)
        area1 = (box1[2] - box1[0]) * (box1[3] - box1[1])
        area2 = (box2[2] - box2[0]) * (box2[3] - box2[1])
        
        # Return intersection ratio instead of IoU for better person-motorcycle association
        return intersection / min(area1, area2)
    
    def expand_box(self, box, frame_shape, expand_ratio=1.3):
        """Expand bounding box to better capture riders"""
        height, width = frame_shape[:2]
        center_x = (box[0] + box[2]) / 2
        center_y = (box[1] + box[3]) / 2
        box_width = (box[2] - box[0]) * expand_ratio
        box_height = (box[3] - box[1]) * expand_ratio
        
        x1 = max(0, center_x - box_width/2)
        y1 = max(0, center_y - box_height/2)
        x2 = min(width, center_x + box_width/2)
        y2 = min(height, center_y + box_height/2)
        
        return [x1, y1, x2, y2]
    
    def process_frame(self, frame):
        """Process a single frame to detect triple riding"""
        # Run YOLOv8 tracking
        results = self.model.track(frame, persist=True, conf=self.CONFIDENCE_THRESHOLD, 
                                 classes=[self.MOTORCYCLE_CLASS, self.PERSON_CLASS])
        
        if results[0].boxes.id is None:
            return frame
            
        # Extract detections
        boxes = results[0].boxes.xyxy.cpu().numpy()
        classes = results[0].boxes.cls.cpu().numpy()
        track_ids = results[0].boxes.id.cpu().numpy().astype(int)
        confidences = results[0].boxes.conf.cpu().numpy()
        
        # Separate motorcycles and persons
        motorcycles = []
        motorcycle_ids = []
        persons = []
        
        for box, cls, track_id, conf in zip(boxes, classes, track_ids, confidences):
            if conf < self.CONFIDENCE_THRESHOLD:
                continue
                
            if cls == self.MOTORCYCLE_CLASS:
                motorcycles.append(box)
                motorcycle_ids.append(track_id)
            elif cls == self.PERSON_CLASS:
                persons.append(box)
        
        # Process each motorcycle
        for motorcycle_idx, (motorcycle_box, motorcycle_id) in enumerate(zip(motorcycles, motorcycle_ids)):
            # Expand motorcycle box to better capture riders
            expanded_box = self.expand_box(motorcycle_box, frame.shape)
            
            # Count riders for this motorcycle
            rider_count = 0
            for person_box in persons:
                overlap = self.calculate_overlap(expanded_box, person_box)
                if overlap > self.IOU_THRESHOLD:
                    rider_count += 1
            
            # Update tracking history
            self.TRACKING_HISTORY[motorcycle_id].append(rider_count)
            if len(self.TRACKING_HISTORY[motorcycle_id]) > self.DETECTION_PERSISTENCE:
                self.TRACKING_HISTORY[motorcycle_id].pop(0)
            
            # Get smoothed rider count
            smooth_count = max(1, round(np.mean(self.TRACKING_HISTORY[motorcycle_id])))
            self.MOTORCYCLE_RIDERS[motorcycle_id] = smooth_count
            
            # Determine color and text based on rider count
            if smooth_count >= 3:
                color = (0, 0, 255)  # Red for triple riding
                text = "TRIPLE RIDING DETECTED!"
            else:
                color = (0, 255, 0)  # Green for normal riding
                text = f"Riders: {smooth_count}"
            
            # Draw annotations
            cv2.rectangle(frame, 
                         (int(motorcycle_box[0]), int(motorcycle_box[1])),
                         (int(motorcycle_box[2]), int(motorcycle_box[3])),
                         color, 2)
            
            # Add text with better visibility
            text_size = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, 0.9, 2)[0]
            cv2.rectangle(frame,
                         (int(motorcycle_box[0]), int(motorcycle_box[1] - text_size[1] - 10)),
                         (int(motorcycle_box[0] + text_size[0]), int(motorcycle_box[1])),
                         color, -1)
            cv2.putText(frame, text,
                       (int(motorcycle_box[0]), int(motorcycle_box[1] - 10)),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 255, 255), 2)
        
        # Clean up old tracks
        current_time = time.time()
        for track_id in list(self.TRACKING_HISTORY.keys()):
            if track_id not in motorcycle_ids:
                del self.TRACKING_HISTORY[track_id]
                if track_id in self.MOTORCYCLE_RIDERS:
                    del self.MOTORCYCLE_RIDERS[track_id]
        
        return frame
    
    def process_video(self, input_path, output_path):
        """Process video file"""
        cap = cv2.VideoCapture(input_path)
        if not cap.isOpened():
            raise ValueError("Error opening video file")
        
        # Get video properties
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        fps = int(cap.get(cv2.CAP_PROP_FPS))
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        
        # Initialize video writer
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
        
        frame_count = 0
        
        try:
            while cap.isOpened():
                ret, frame = cap.read()
                if not ret:
                    break
                
                frame_count += 1
                if frame_count % 30 == 0:
                    print(f"Processing frame {frame_count}/{total_frames}")
                
                # Process frame
                processed_frame = self.process_frame(frame)
                
                # Write and display frame
                out.write(processed_frame)
                
                # Display frame (optional)
                cv2.imshow('Triple Rider Detection', cv2.resize(processed_frame, (1280, 720)))
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break
                
        finally:
            cap.release()
            out.release()
            cv2.destroyAllWindows()

def main():
    # Initialize detector with YOLOv8x for best accuracy
    detector = TripleRiderDetector(model_path="yolov8x.pt")
    
    # Process video
    input_video = "C:/Users/DELL/Desktop/accident detection/numberplate/triplesamplevideo.mp4"  # Replace with your video path
    output_video = "C:/Users/DELL/Desktop/accident detection/numberplate/triplesamplevideooutput.mp4"
    
    detector.process_video(input_video, output_video)

if __name__ == "__main__":
    main()
'''
#same on the image code:
import cv2
import numpy as np
from ultralytics import YOLO
import torch
from collections import defaultdict

class TripleRiderDetector:
    def __init__(self, model_path="yolov8x.pt"):
        # Initialize YOLOv8 model
        self.model = YOLO(model_path)
        
        # Constants
        self.MOTORCYCLE_CLASS = 3  # COCO class for motorcycle
        self.PERSON_CLASS = 0      # COCO class for person
        self.CONFIDENCE_THRESHOLD = 0.5
        self.IOU_THRESHOLD = 0.3
    
    def calculate_overlap(self, box1, box2):
        """Calculate IoU between two bounding boxes"""
        x1 = max(box1[0], box2[0])
        y1 = max(box1[1], box2[1])
        x2 = min(box1[2], box2[2])
        y2 = min(box1[3], box2[3])
        
        intersection = max(0, x2 - x1) * max(0, y2 - y1)
        area1 = (box1[2] - box1[0]) * (box1[3] - box1[1])
        area2 = (box2[2] - box2[0]) * (box2[3] - box2[1])
        
        return intersection / min(area1, area2)
    
    def expand_box(self, box, frame_shape, expand_ratio=1.3):
        """Expand bounding box to better capture riders"""
        height, width = frame_shape[:2]
        center_x = (box[0] + box[2]) / 2
        center_y = (box[1] + box[3]) / 2
        box_width = (box[2] - box[0]) * expand_ratio
        box_height = (box[3] - box[1]) * expand_ratio
        
        x1 = max(0, center_x - box_width/2)
        y1 = max(0, center_y - box_height/2)
        x2 = min(width, center_x + box_width/2)
        y2 = min(height, center_y + box_height/2)
        
        return [x1, y1, x2, y2]
    
    def process_image(self, image_path, output_path=None):
        """Process a single image to detect triple riding"""
        # Read image
        frame = cv2.imread(image_path)
        if frame is None:
            raise ValueError("Error reading image file")
        
        # Run YOLOv8 detection
        results = self.model(frame, conf=self.CONFIDENCE_THRESHOLD, 
                           classes=[self.MOTORCYCLE_CLASS, self.PERSON_CLASS])
        
        # Extract detections
        boxes = results[0].boxes.xyxy.cpu().numpy()
        classes = results[0].boxes.cls.cpu().numpy()
        confidences = results[0].boxes.conf.cpu().numpy()
        
        # Separate motorcycles and persons
        motorcycles = []
        persons = []
        
        for box, cls, conf in zip(boxes, classes, confidences):
            if conf < self.CONFIDENCE_THRESHOLD:
                continue
                
            if cls == self.MOTORCYCLE_CLASS:
                motorcycles.append(box)
            elif cls == self.PERSON_CLASS:
                persons.append(box)
        
        # Process each motorcycle
        for motorcycle_box in motorcycles:
            # Expand motorcycle box to better capture riders
            expanded_box = self.expand_box(motorcycle_box, frame.shape)
            
            # Count riders for this motorcycle
            rider_count = 0
            for person_box in persons:
                overlap = self.calculate_overlap(expanded_box, person_box)
                if overlap > self.IOU_THRESHOLD:
                    rider_count += 1
            
            # Determine color and text based on rider count
            if rider_count >= 3:
                color = (0, 0, 255)  # Red for triple riding
                text = "TRIPLE RIDING DETECTED!"
            else:
                color = (0, 255, 0)  # Green for normal riding
                text = f"Riders: {rider_count}"
            
            # Draw annotations
            cv2.rectangle(frame, 
                         (int(motorcycle_box[0]), int(motorcycle_box[1])),
                         (int(motorcycle_box[2]), int(motorcycle_box[3])),
                         color, 2)
            
            # Add text with better visibility
            text_size = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, 0.9, 2)[0]
            cv2.rectangle(frame,
                         (int(motorcycle_box[0]), int(motorcycle_box[1] - text_size[1] - 10)),
                         (int(motorcycle_box[0] + text_size[0]), int(motorcycle_box[1])),
                         color, -1)
            cv2.putText(frame, text,
                       (int(motorcycle_box[0]), int(motorcycle_box[1] - 10)),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 255, 255), 2)
        
        # Save or display the result
        if output_path:
            cv2.imwrite(output_path, frame)
        
        # Display the result
        cv2.imshow('Triple Rider Detection', cv2.resize(frame, (1280, 720)))
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        
        return frame

def main():
    # Initialize detector
    detector = TripleRiderDetector(model_path="yolov8x.pt")
    
    # Process image
    input_image = "C:/Users/DELL/Desktop/accident detection/numberplate/tripleimage.jpg"  # Replace with your image path
    output_image = "C:/Users/DELL/Desktop/accident detection/numberplate/tripleimageoutput.jpg"  # Replace with desired output path
    
    detector.process_image(input_image, output_image)

if __name__ == "__main__":
    main()
'''
'''so this code gives you the output as the footage video of the triple riding video by intput as the video from the CCTV camera. 
the video first is extracted frame by frame ( so images ), now this images are passed through yolov8x (biggest model) and then this model tries to detect the vehicle, and person on the vehicle.
now, if the vehicle is detected with onlt one person is ok, but, if the persons on the vehicle (i.e motorcycle) are more then 2 (for example 3) then it considers as a triple riding.
this code works with images and videos. in the above code we have two sections 1: with images, 2: with video. '''

In [ ]:
#triple riding and number plate 
import cv2
import numpy as np
from ultralytics import YOLO
import easyocr
from datetime import datetime
import os

def create_folders():
    # Create folders if they don't exist
    if not os.path.exists('violations'):
        os.makedirs('violations')

def calculate_overlap(box1, box2):
    """Calculate IOU between two boxes"""
    x1 = max(box1[0], box2[0])
    y1 = max(box1[1], box2[1])
    x2 = min(box1[2], box2[2])
    y2 = min(box1[3], box2[3])
    
    if x2 < x1 or y2 < y1:
        return 0.0
        
    intersection = (x2 - x1) * (y2 - y1)
    box1_area = (box1[2] - box1[0]) * (box1[3] - box1[1])
    
    if box1_area == 0:
        return 0.0
        
    return intersection / box1_area

def main():
    # Initialize YOLOv8 model
    model = YOLO('yolov8x.pt')  # or use 'yolov8x.pt' for better accuracy
    
    # Initialize EasyOCR
    reader = easyocr.Reader(['en'])
    
    # Create output folder
    create_folders()
    
    # Open video capture
    #video_path = 'C:/Users/DELL/Desktop/accident detection/yolo11-speed-data-save-main/video2.mp4' 
    video_path = 'C:/Users/DELL/Desktop/accident detection/numberplate/tripletest.mp4'
     # Use 0 for webcam or provide video file path
    cap = cv2.VideoCapture(video_path)
    
    while cap.isOpened():
        success, frame = cap.read()
        if not success:
            break
            
        # Run YOLOv8 detection
        results = model(frame)
        
        # Get motorcycles and persons
        motorcycles = []
        persons = []
        
        # Process detections
        for r in results:
            boxes = r.boxes
            for box in boxes:
                # Get box coordinates
                x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
                x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
                
                # Get class name
                cls = int(box.cls[0])
                class_name = model.names[cls]
                
                if class_name == 'motorcycle':
                    motorcycles.append([x1, y1, x2, y2])
                elif class_name == 'person':
                    persons.append([x1, y1, x2, y2])
        
        # Process each motorcycle
        for motorcycle in motorcycles:
            # Count people on this motorcycle
            people_count = 0
            for person in persons:
                overlap = calculate_overlap(motorcycle, person)
                if overlap > 0.3:  # If person overlaps with motorcycle by 30%
                    people_count += 1
            
            # Draw motorcycle box
            color = (0, 0, 255) if people_count > 2 else (0, 255, 0)
            cv2.rectangle(frame, 
                         (motorcycle[0], motorcycle[1]), 
                         (motorcycle[2], motorcycle[3]), 
                         color, 2)
            
            # Add count text
            cv2.putText(frame, 
                       f'Riders: {people_count}', 
                       (motorcycle[0], motorcycle[1] - 10),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
            
            # Process violations (more than 2 people)
            if people_count > 2:
                # Crop motorcycle region
                crop_img = frame[motorcycle[1]:motorcycle[3], 
                               motorcycle[0]:motorcycle[2]]
                
                # Try to read number plate
                try:
                    plate_results = reader.readtext(crop_img)
                    plate_text = ""
                    for (bbox, text, prob) in plate_results:
                        if len(text) > 3 and any(c.isdigit() for c in text):
                            plate_text = text
                            break
                    
                    # Save violation image
                    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
                    filename = f"violations/violation_{timestamp}_riders{people_count}"
                    if plate_text:
                        filename += f"_plate{plate_text}"
                    filename += ".jpg"
                    
                    cv2.imwrite(filename, crop_img)
                    
                    # Add plate text to frame
                    if plate_text:
                        cv2.putText(frame,
                                  f'Plate: {plate_text}',
                                  (motorcycle[0], motorcycle[3] + 20),
                                  cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
                except Exception as e:
                    print(f"Error processing number plate: {e}")
        
        # Show frame
        cv2.imshow('Motorcycle Rider Detection', frame)
        
        # Break loop on 'q' press
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    # Cleanup
    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()